In [24]:
import numpy as np
from os import makedirs, listdir
from os.path import join
from utils.ZR_utils import gen_proj, normalize_dimensions_nodiv0, std_to_lsh64


In [4]:
%load_ext cython

In [5]:
%%cython
import numpy as np

def std_to_lsh64_c(int D, float[::1] feat_std, float[::1] daps_projmat ):

    cdef int S = 32
    cdef int n_frames = int(len(feat_std)) / D
    cdef int[::1] sigs = np.zeros((n_frames*2,1),dtype=np.uint32)

    cdef int i,b,d
    cdef float proj = 0

    for i in range(n_frames):

        for b in range(S):
            proj = 0
            for d in range(D):
                proj += feat_std[i*D + d] * daps_projmat[b*D + d]
            sigs[2*i] += (proj>0) * 2**b

        for b in range(S):
            proj = 0
            for d in range(D):
                proj += feat_std[i*D + d] * daps_projmat[(b+S)*D + d]
            sigs[2*i + 1] += (proj>0) * 2**b       

    return sigs


In [23]:
npy_path = '/home/korhan/Dropbox/tez/features/Signer06/deep_hand/l3/right/train'
exp_name = '_'.join('Signer06/deep_hand/l3/right'.split('/'))

files = listdir(npy_path)
print('{} files are found for experiment \n{}'.format(len(files), exp_name))

30 files are found for experiment 
Signer06_deep_hand_l3_right


In [18]:
from os import makedirs
dst_path = '/home/korhan/Dropbox/tez/features/to_zrtools/lsh'
makedirs(join(dst_path,exp_name), exist_ok=True)

In [19]:
projmatpath = '/home/korhan/Desktop/zerospeech2017/track2/src/ZRTools/exp/Signer06_deep_hand_l1_right/proj_S64xD1024_seed1'
projmat = np.fromfile(projmatpath, dtype=np.float32)
print(projmat.shape)

(65536,)


In [20]:
text_file = open(join(dst_path, '..', exp_name + '.lsh64.lst'), 'w')

for filename in files:

    array2d = np.load(join(npy_path,filename))

    array2d_norm = normalize_dimensions_nodiv0(array2d,axis=0)    
#    array2d_norm = array2d

    array1d = convert_2d_to_1d(array2d_norm, dtype=np.float32)

    sigs = std_to_lsh64(D=1024, feat_std=array1d, projmat=projmat)

    sigs.tofile(join(dst_path, exp_name, filename[:-4] + '.std.lsh64'))
    
    text_file.writelines( join( dst_path, exp_name, filename[:-4] + '.std.lsh64\n') )

text_file.close()
    